In [ ]:
import multiprocessing as mp
from glass.pys.oss import lst_ff


import os
from glass.pys.oss import mkdir, fprop
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str

from glass.it.shp import grs_to_shp
from glass.it.rst import rst_to_grs

In [ ]:
#refs = '/home/francisco/create_grid/multiprocess/rasters_test'
refs = '/mnt/d/autocls01/dadosfrancisco/multiprocess/rasters_test/'

#out_grid = '/home/francisco/create_grid/multiprocess/grids'
out_grid = '/mnt/d/autocls01/dadosfrancisco/multiprocess/grids'

bname = 'terceira_fish'
nproc = 2

In [ ]:
rsts = lst_ff(refs, file_format='.tif')

In [ ]:
rsts

In [ ]:
dimlist = len(rsts) / nproc

dimlist = int(dimlist) if dimlist == int(dimlist) else int(dimlist) + 1

In [ ]:
process_data = []
for e in range(0, nproc*dimlist, dimlist):
    process_data.append(rsts[e:e+dimlist])

In [ ]:
process_data

In [ ]:
def create_fishnet(th, idx, raster, out_shp):
    #print(f'Process {str(th)} - {raster}')
    
    # Check if outfolder exists
    out_folder = os.path.dirname(out_shp)
    if not os.path.exists(out_folder):
        mkdir(out_folder, overwrite=None)

    """
    Start GRASS GIS Session
    """
    
    loc = f'loc_{now_as_str()}_{str(th)}_{str(idx)}' 
    #loc = f'loc_{now_as_str()}'
    grsb = run_grass(
        out_folder, grassBIN='grass78', location=loc,
        srs=raster
    )
    
    import grass.script.setup as gsetup
    
    gsetup.init(grsb, out_folder, loc, 'PERMANENT')
    
    from grass.pygrass.modules import Module
    
    bloc = rst_to_grs(raster, fprop(raster, 'fn'))
    
    grsgrid = fprop(out_shp, 'fn')
    
    m = Module('v.mkgrid', map=grsgrid, overwrite=True, run_=True, quiet=True)
    
    out_grid = grs_to_shp(grsgrid, out_shp, 'area')
    
def run_multiproc(th, rasters, out_grid, name):
    for _r in range(len(rasters)):
        create_fishnet(
            th, _r, rasters[_r],
            os.path.join(out_grid, f'{name}_{str(th)}_{str(_r)}.shp'))

thrds = []

for i in range(len(process_data)):
    processo = mp.Process(
        target=run_multiproc, name=f'th-{str(i+1)}',
        args=(i+1, process_data[i], out_grid, bname)
    )

    thrds.append(processo)

for p in thrds:
    p.start()

for p in thrds:
    p.join()

print('terminou')